import pymysql 
import pandas as pd #Python

In [3]:
import pymysql 
myConnection = pymysql.connect(host='bk3015.mysql.pythonanywhere-services.com', 
                               user='bk3015', 
                               passwd='los12345', 
                               db='bk3015$los')




OperationalError: (2003, "Can't connect to MySQL server on 'bk3015.mysql.pythonanywhere-services.com' (timed out)")

In [4]:
import pandas as pd #Python tool for data manipulation and analysis. The two primary data structures are Series and DataFrame 
import numpy as np

In [5]:
cur = myConnection.cursor()
cur.execute("SELECT * FROM patients")
myres = cur.fetchall()
labels = [x[0] for x in cur.description] 
patients = pd.DataFrame(list(myres), columns = labels)
patients.to_csv('patients.csv')

NameError: name 'myConnection' is not defined

In [6]:
data = pd.read_csv('patients.csv', index_col=0, na_values=(" ","  ","   ","    ","     "), low_memory=False)
data.columns = data.columns.str.strip()
# print first 10 rows
data.shape

(3900, 6)

In [7]:
data['ins'] = data['INSURANCE']
data['mar'] = data['MARITAL']
data['lang'] = data['LANGUAGE']
data['race'] = data['RACE']
data.head()

,PATIENT_ID,INSURANCE,RACE,LANGUAGE,MARITAL,LOS,ins,mar,lang,race
0,2,Private,ASIAN,NaN,NaN,4,Private,NaN,NaN,ASIAN
1,3,Medicare,WHITE,NaN,MARRIED,11,Medicare,MARRIED,NaN,WHITE
2,4,Private,WHITE,NaN,SINGLE,7,Private,SINGLE,NaN,WHITE
3,5,Private,ASIAN,NaN,NaN,2,Private,NaN,NaN,ASIAN
4,6,Medicare,WHITE,ENGL,MARRIED,16,Medicare,MARRIED,ENGL,WHITE


In [8]:
ins_type = data.INSURANCE.unique().tolist()
ins_type = [x for x in ins_type if str(x) != 'nan']
print(ins_type)

race_type = data.RACE.unique().tolist()
race_type = [x for x in race_type if str(x) != 'nan']
print(race_type)
lang_type = data.LANGUAGE.unique().tolist()
lang_type = [x for x in lang_type if str(x) != 'nan']
print(lang_type)
mar_type = data.MARITAL.unique().tolist()
mar_type = [x for x in mar_type if str(x) != 'nan']
print(mar_type)

['Private', 'Medicare', 'Medicaid', 'Government', 'Self Pay']
['ASIAN', 'WHITE', 'UNKNOWN/NOT SPECIFIED', 'BLACK/AFRICAN AMERICAN', 'PATIENT DECLINED TO ANSWER', 'OTHER', 'HISPANIC OR LATINO', 'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - PUERTO RICAN', 'ASIAN - ASIAN INDIAN', 'ASIAN - VIETNAMESE', 'MULTI RACE ETHNICITY', 'HISPANIC/LATINO - DOMINICAN', 'AMERICAN INDIAN/ALASKA NATIVE', 'WHITE - RUSSIAN', 'HISPANIC/LATINO - SALVADORAN', 'ASIAN - CHINESE', 'UNABLE TO OBTAIN', 'BLACK/CAPE VERDEAN', 'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'BLACK/HAITIAN', 'SOUTH AMERICAN']
['ENGL', 'MAND', 'CANT', 'SPAN', 'GREE', 'CAMB', '*ARM', 'HAIT', 'RUSS', 'CAPE', 'PORT', '*MAN', 'VIET', 'ITAL', 'HIND', 'PTUN', '*CHI', 'ALBA', 'ARAB', 'POLI', 'FREN', 'AMER', 'LAOT', '*URD', '*PUN', '*BUR', '*BEN']
['MARRIED', 'SINGLE', 'WIDOWED', 'DIVORCED', 'SEPARATED', 'UNKNOWN (DEFAULT)']


In [9]:
new_data = data.copy()

#we are doing this the long way 
ins_dict= {}
race_dict = {}
lang_dict = {}
mar_dict = {}
for i in range(len(ins_type)):
    ins_dict[ins_type[i]] = i; 
for i in range(len(race_type)):
    race_dict[race_type[i]] = i; 
for i in range(len(lang_type)):
    lang_dict[lang_type[i]] = i; 
for i in range(len(mar_type)):
    mar_dict[mar_type[i]] = i; 

new_data.ins = [ins_dict[item] for item in new_data.ins]
new_data.race = [race_dict[item] for item in new_data.race]
#new_data.lang = [lang_dict[item] for item in new_data.lang]
#new_data.mar = [mar_dict[item] for item in new_data.mar]

for index, row in new_data.iterrows():
    if(str(row['mar']) == 'nan' and str(row['lang']) == 'nan'):
        continue 
    elif(str(row['lang']) == 'nan' and  not(str(row['mar']) == 'nan')):
        new_data.at[index, 'mar'] = mar_dict[row['mar']]
        continue 
    if(not(str(row['lang']) == 'nan') and str(row['mar']) == 'nan'):
        new_data.at[index, 'lang'] = lang_dict[row['lang']]
        continue 
    new_data.at[index, 'mar'] = mar_dict[row['mar']]
    new_data.at[index, 'lang'] = lang_dict[row['lang']]

sub_data = new_data.drop(columns = ['INSURANCE','RACE', 'LANGUAGE', 'MARITAL'])
sub_data.head()

cols = list(sub_data.columns.values)



In [10]:
print(mar_dict)

{'MARRIED': 0, 'SINGLE': 1, 'WIDOWED': 2, 'DIVORCED': 3, 'SEPARATED': 4, 'UNKNOWN (DEFAULT)': 5}


In [11]:
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(sub_data)
imp_data = pd.DataFrame(imp.transform(sub_data.values))
#cols = ['EOD10_EX', 'EOD10_PN','EOD10_NE', 'RADIATN', 'GRADE','SEQ_NUM', 'RAC_RECA', 'AGE_DX', 'srv_time_mon', 'alive']
imp_data.columns = cols
imp_data.head()

c:\users\dfran\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


,PATIENT_ID,LOS,ins,mar,lang,race
0,2.0,4.0,0.0,0.848897,2.318117,0.0
1,3.0,11.0,1.0,0.000000,2.318117,1.0
2,4.0,7.0,0.0,1.000000,2.318117,1.0
3,5.0,2.0,0.0,0.848897,2.318117,0.0
4,6.0,16.0,1.0,0.000000,0.000000,1.0


In [12]:
imp_data.tail()

,PATIENT_ID,LOS,ins,mar,lang,race
3895,5952.0,44.0,2.0,4.000000,3.000000,6.0
3896,5953.0,6.0,0.0,0.000000,2.318117,1.0
3897,5954.0,4.0,0.0,1.000000,0.000000,1.0
3898,5955.0,28.0,1.0,3.000000,2.318117,1.0
3899,5956.0,2.0,3.0,0.848897,2.318117,3.0


In [13]:
to_split =  imp_data
test_data = to_split.iloc[:3000, :]
train_data = to_split.iloc[3000:, :]

X_test  = test_data.iloc[:, 2:]
X_train =  train_data.iloc[:, 2:]

y1_train = train_data.iloc[:, 1:2]# YOUR CODE HERE
y1_test  = test_data.iloc[:, 1:2]# YOUR CODE HERE


In [19]:
import matplotlib.pyplot as plt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
regr = linear_model.LinearRegression()
#regr = ElasticNet(random_state=0)
#regr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
# Train the model using the training sets
regr.fit(X_train, y1_train)
#maital race language insurnace 

# Make predictions using the testing set
y1_pred = regr.predict(X_test)
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y1_test, y1_pred))

#Print the intercept and coefficients of the resulting model.
print (regr.intercept_)
print (regr.coef_)

from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score
print("Mean squared error: %.2f"
      % mean_squared_error(y1_test, y1_pred))
print("root mean squared error: %.2f"
      % sqrt(mean_squared_error(y1_test, y1_pred)))

Coefficients: 
 [[ 0.46761419  0.33009392 -0.07575373 -0.04153496]]
Variance score: -0.00
[10.07758311]
[[ 0.46761419  0.33009392 -0.07575373 -0.04153496]]
Mean squared error: 211.86
root mean squared error: 14.56


In [20]:
import numpy as np
enum_list = []
a = ['PATIENT_ID', 'INSURANCE', 'RACE', 'LANGUAGE', 'MARITAL', 'LOS']
counter = 5957
for i in lang_type:
    for j in race_type:
        for m in ins_type:
            for n in mar_type:
                d = dict.fromkeys(a, 0)
                d['PATIENT_ID'] = counter
                d['INSURANCE'] = m
                d['RACE'] = j
                d['LANGUAGE'] = i
                d['MARITAL'] = n
                d['LOS'] = 0
                enum_list.append(d)
                counter+=1
                
mat = []
for i in enum_list:
    
    lst = []
    lst = [None] * 4
    play=list(i.values())
    lst[0] = ins_dict[play[1]] 
    lst[1] = race_dict[play[2]] 
    lst[2] = lang_dict[play[3]]
    lst[3] = mar_dict[play[4]]
    mat.append(lst)



In [21]:
q = regr.predict(np.array(mat))
for i in range(len(enum_list)):
    enum_list[i]['LOS'] = q[i]
    
print(enum_list)

[{'PATIENT_ID': 5957, 'INSURANCE': 'Private', 'RACE': 'ASIAN', 'LANGUAGE': 'ENGL', 'MARITAL': 'MARRIED', 'LOS': array([10.07758311])}, {'PATIENT_ID': 5958, 'INSURANCE': 'Private', 'RACE': 'ASIAN', 'LANGUAGE': 'ENGL', 'MARITAL': 'SINGLE', 'LOS': array([10.03604815])}, {'PATIENT_ID': 5959, 'INSURANCE': 'Private', 'RACE': 'ASIAN', 'LANGUAGE': 'ENGL', 'MARITAL': 'WIDOWED', 'LOS': array([9.99451319])}, {'PATIENT_ID': 5960, 'INSURANCE': 'Private', 'RACE': 'ASIAN', 'LANGUAGE': 'ENGL', 'MARITAL': 'DIVORCED', 'LOS': array([9.95297823])}, {'PATIENT_ID': 5961, 'INSURANCE': 'Private', 'RACE': 'ASIAN', 'LANGUAGE': 'ENGL', 'MARITAL': 'SEPARATED', 'LOS': array([9.91144327])}, {'PATIENT_ID': 5962, 'INSURANCE': 'Private', 'RACE': 'ASIAN', 'LANGUAGE': 'ENGL', 'MARITAL': 'UNKNOWN (DEFAULT)', 'LOS': array([9.86990831])}, {'PATIENT_ID': 5963, 'INSURANCE': 'Medicare', 'RACE': 'ASIAN', 'LANGUAGE': 'ENGL', 'MARITAL': 'MARRIED', 'LOS': array([10.5451973])}, {'PATIENT_ID': 5964, 'INSURANCE': 'Medicare', 'RACE':

In [22]:
import csv
toCSV = enum_list
keys = a
with open('people.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)